### Trabalho realizado pelo grupo G18

- #### José Silva A100105
- #### Alexandra Calafate A100060


# TP3 - Problema 2:


1. Construa um “Control Flow Automaton (CFA)” que determina este programa. Identifique os locais e as transições/ramos. Numa abordagem orientada às pré-condições identifique os transformadores de predicados associados aos vários locais e os “switches” associados aos vários ramos.

2. Construa em `z3` o sistema de equações que representa o comportamento deste sistema dinâmico sob o ponto de vista da prova de segurança e verifique a segurança do programa através da resolução (total ou parcial) deste sistema. > sugere-se (não é obrigatório mas é valorizado !), na alínea (a), uma representação do CFA através de um grafo orientado implementado em `networkx` e a sua compilação para o sistema de equações.


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Criar o grafo dirigido
G = nx.DiGraph()

# Nós do grafo: identificação e texto dentro das bolinhas
nodes = [
    ("system", "havoc(a, b)"),
    ("check_rlinha", "skip"),
    ("process", "q = r div rlinha\n"
                "r, rlinha, s, slinha, t, tlinha =\n"
                "rlinha, r - q × rlinha,\n"
                "slinha, s - q × slinha,\n"
                "tlinha, t - q × tlinha"),
    ("stop", "skip"),
    ("error", "skip"),
]

# Transições do grafo: origem, destino, rótulo das arestas (se necessário)
edges = [
    ("system", "check_rlinha", "r, rlinha, s, slinha, t, tlinha = a, b, 1, 0, 0, 1"),
    ("check_rlinha", "process", "rlinha != 0"),
    ("check_rlinha", "stop", "rlinha == 0"),
    ("process", "check_rlinha", "¬Overflow"),  # Corrigido para ¬Overflow
    ("process", "error", "overflow"),
]

# Adicionar nós ao grafo
for node_id, label in nodes:
    G.add_node(node_id, label=label)

# Adicionar arestas ao grafo
for source, target, *edge_label in edges:
    label = edge_label[0] if edge_label else ""
    G.add_edge(source, target, label=label)

# Gerar o layout hierárquico (top-down) com o PyGraphviz
pos = nx.nx_agraph.graphviz_layout(G, prog="dot")

# Configurar o desenho do grafo
plt.figure(figsize=(12, 10))
nx.draw(
    G,
    pos=pos,
    with_labels=True,
    labels={node: G.nodes[node]['label'] for node in G.nodes},  # Usar os textos como rótulos
    node_size=10000,
    node_color="lightblue",
    font_size=9,
    font_weight="bold",
    connectionstyle="arc3,rad=0.1",
)

# Adicionar rótulos às arestas com texto ajustado manualmente
edge_labels = nx.get_edge_attributes(G, "label")

# Adicionar rótulos de forma ajustada para evitar sobreposição
for (source, target), label in edge_labels.items():
    x_offset = (pos[target][0] - pos[source][0]) / 2
    y_offset = (pos[target][1] - pos[source][1]) / 2
    label_x = pos[source][0] + x_offset
    label_y = pos[source][1] + y_offset

    # Ajuste de deslocamento específico para evitar sobreposição
    if label == "rlinha != 0":
        label_x -= 20  # Deslocar mais para a esquerda
    elif label == "¬Overflow":
        label_x += 20  # Deslocar mais para a direita

    plt.text(label_x, label_y, label, fontsize=8, ha="center", va="center")

# Título do gráfico
plt.title("Programa Representado Como Fluxo", fontsize=14)
plt.show()


In [ ]:
from pysmt.shortcuts import *

def graph_safety(a, b, n):
    print(f"> A testar {n} iterações.")
    
    with Solver(name = "z3") as solver:
        
        r = Symbol("r", INT)          # Variável simbólica r
        rlinha = Symbol("rlinha", INT) # Variável simbólica r'
        s = Symbol("s", INT)          # Variável simbólica s
        slinha = Symbol("slinha", INT) # Variável simbólica s'
        t = Symbol("t", INT)          # Variável simbólica t
        tlinha = Symbol("tlinha", INT) # Variável simbólica t'

        # Inicialização das variáveis de controle
        stop = FALSE()   # Estado de paragem inicializado como falso
        do = FALSE()     # Variável de controle do loop inicializada como falso

        # Define as condições iniciais do sistema
        var = And(
            Equals(r, Int(a)), Equals(rlinha, Int(b)),  # r = a, r' = b
            Equals(s, Int(0)), Equals(slinha, Int(1)),  # s = 0, s' = 1
            Equals(t, Int(1)), Equals(tlinha, Int(0))   # t = 1, t' = 0
        )

        # Loop principal que verifica n iterações
        for i in range(n):
            # Combina as condições iniciais com o estado atual
            system = And(var, do)
            
            # Verifica se o sistema tem solução
            if solver.is_sat(system):
                print(f"> Iteração {i}: sistema inseguro.")
                return

            # Cálculo do quociente
            q = Div(r, rlinha)
            
            # Atualização das variáveis usando substituição
            atrib = substitute(do, {
                r: rlinha,                          # r ← r'
                rlinha: Minus(r, Times(q, rlinha)), # r' ← r - q × r'
                s: slinha,                          # s ← s'
                slinha: Minus(s, Times(q, slinha)), # s' ← s - q × s'
                t: tlinha,                          # t ← t'
                tlinha: Minus(t, Times(q, tlinha))  # t' ← t - q × t'
            })

            # Define as novas condições do sistema
            new_do = And(
                Implies(Equals(rlinha, Int(0)), stop),           # Se r' = 0, então para
                Implies(Not(Equals(rlinha, Int(0))), atrib)      # Se r' ≠ 0, então continua com as atribuições
            )
            
            # Atualiza a variável de controle
            do = Or(do, new_do)

        # Se chegou aqui, não encontrou estados inseguros
        print("> Não foram encontrados estados inseguros.")

# Testa o algoritmo com valores iniciais a=8, b=4 e 10 iterações
graph_safety(8, 4, 10)